In [1]:
import pandas as pd
import urllib
import os
import string
import re
import numpy as np

In [2]:
df = pd.read_csv("Data/voies_paris_labelisé.csv", sep=';', index_col=0)
df.head()

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,Personne,...,profession,profession2,profession3,LARGEUR,LONG,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN
0,1864-02-03T00:53:28+00:53,4166,Roquette.,rue,NaN,gerbier,rue Gerbier,4112,"Pierre Jean Baptiste Gerbier (1725-1788), avoc...",Pierre Jean Baptiste Gerbier,...,avocat,NaN,NaN,13.0,174.0,Ouverture et alignements (non retenus au POS e...,XIe,voie publique,"rue de la Folie-Regnault, 15.","rue de la Roquette, 168 bis."
1,NaN,4994,Saint-Ambroise.,cité,NaN,joly,cité Joly,4886,"M. Joly, propriétaire.",M. Joly,...,propriétaire,NaN,NaN,12.0,135.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"rue du Chemin Vert, 121.",en impasse
2,NaN,5441,Saint-Ambroise.,rue,NaN,léchevin,rue Léchevin,5376,"M. Léchevin, propriétaire.\nHistorique. ~ Avan...",M. Léchevin,...,propriétaire,NaN,NaN,15.0,73.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"avenue Parmentier, 64.","passage Saint-Ambroise, 9 bis."
3,NaN,8208,Folie-Méricourt. Saint-Ambroise. Roquette.,boulevard,NaN,richard lenoir,boulevard Richard Lenoir,8197,"François Richard (1765-1839), manufacturier d'...",François Richard,...,manufacturier,NaN,NaN,60.0,1500.0,Alignements (non retenus au POS et non repris ...,XIe,voie publique,"boulevard Beaumarchais, 2 et place de la Basti...","avenue de la République, 22, rue Rampon et pla..."
4,1885-10-11T00:53:28+00:53,9057,Saint-Ambroise.,rue,NaN,spinoza,rue Spinoza,8671,"Baruch Spinoza (1632-1677), philosophe holland...",Baruch Spinoza,...,philosophe,NaN,NaN,12.0,77.0,"Ouverture, alignements (non retenus au POS et ...",XIe,voie publique,"avenue de la République, 103.","boulevard de Ménilmontant, 81."


In [3]:
len(df)

6475

# 1. Clean "HISTORIQUE"

On va utiliser la variable "HISTORIQUE", pour extraire des informations relatives à la personne qui a donné son nom à la voie.

In [4]:
def cleaning_historique() :
    df['Libellé VOIE'] = df['Libellé VOIE'].astype(str).apply(lambda x: x.replace('-', ' '))
    df['Libellé VOIE'] = df['Libellé VOIE'].astype(str).apply(lambda x: x.replace('et ', ''))
    
    # Removing expressions "Origin. du nom"
    removed_expressions = ["Orig. du nom. ~ ", "Orig.du nom.~ ", "orig. du nom. ~", "orig. du nom.~ ", "Orig. Du nom.~", "Orig. Du nom. ~"]
    for expression in removed_expressions :
        df["HISTORIQUE"] = df['HISTORIQUE'].astype(str).apply(lambda x: x.replace(expression, ''))

In [5]:
cleaning_historique()

# 2. Voisinages

On distingue les rues qui portent des noms d'individus et les autres.

Plusieurs façons de reconnaître les rues qui portent des noms d'individus :
- On a souvent les dates de naissance et mort entre parenthèses pour les rues qui portent des noms de personnes
- S'il s'agit d'un propriétaire (qui possédait donc des immeubles dans cette rue), on peut filter avec "propriétaire"
- Pour certaines rues (plutôt des impasses), on n'a pas la description de la personne, car il s'agit d'une rue au voisinage d'un boulevard par exemple (et l'information sur la personne est disponible à la ligne correspondant au boulevard)
- Ou sinon, si on n'a ni l'un ni l'autre, on peut utiliser la liste des prénoms (puisque pour certains individus, on n'a ni les autres informations ci-dessus)

In [7]:
# dataframe voisinages: on remplace la "Rues" par la vraie référence à la rue
voisinages = df.loc[(df['HISTORIQUE'].str.startswith("Voisinage")) | (df['HISTORIQUE'].str.startswith("Voir")) | (df['Libellé VOIE'] == 'boulevard Carnot')].reset_index()
voisinages["Voisinage rue"] = voisinages["HISTORIQUE"]
voisinages = pd.merge(voisinages.loc[:, voisinages.columns != 'HISTORIQUE'], 
                      df.loc[(~df['HISTORIQUE'].str.startswith("Voisinage")) & (~df['HISTORIQUE'].str.startswith("Voir")), ['HISTORIQUE', 'NOM de VOIE']], 
                      how='left', 
                      on='NOM de VOIE')

In [8]:
voisinages_dup = voisinages[voisinages['Libellé VOIE'].duplicated(keep=False)]
voisinages_dup['NOM de VOIE'].unique()

array(['arts', 'verdun', 'ménilmontant', 'hôtel de ville', 'tournelle',
       'ermitage', 'terrasse', 'bercy', 'saint-denis', 'monceau',
       'gravelle', 'invalides', 'gravilliers', 'chapelle', 'carnot',
       'saint-michel', 'rome', 'austerlitz', 'saint-paul', 'louvre',
       'ranelagh', 'clichy', 'longchamp', 'havre', 'saint-jacques',
       'saint-mandé', 'bel air', 'saint-fargeau', 'saint-charles',
       'saint-martin', 'caire', 'parc', 'saint-eustache', 'iéna',
       'porte de vanves', 'reuilly', 'vendôme', 'peupliers'], dtype=object)

In [9]:
#On corrige à la main les doublons pour les personnes ; pour le reste, on prendra juste le 1er.
def correction_main(personnes, type_de_voie):
    for personne in personnes:
        df.loc[
            (df['NOM de VOIE']==personne), 'HISTORIQUE'] = df.loc[
            (df['NOM de VOIE']==personne) & (df['TYPE DE VOIE']==type_de_voie), 'HISTORIQUE'].values

correction_main(['anatole france', 'molière', 'carnot'], 'avenue')
correction_main(['voltaire'], 'boulevard')
correction_main(['chartres', 'orléans'], 'galerie')
correction_main(['falguière', 'dauphine', 'sorbonne'], 'rue')
correction_main(['montebello'], 'quai')
# exception car il y a 2 montmorency :
df.loc[df['Libellé VOIE']=='avenue de Montmorency', 'HISTORIQUE'] = df.loc[df['Libellé VOIE']=='boulevard de Montmorency', 'HISTORIQUE'].values

In [10]:
# On a corrigé directement dans le df, donc on supprime ces valeurs de voisinages
voisinages = voisinages[~voisinages['NOM de VOIE'].isin(['anatole france', 'molière', 'carnot', 'voltaire', 'chartres', 'orléans', 'falguière', 'dauphine', 'sorbonne', 'montebello', 'montmorency'])]

# Suppression doublons 
voisinages = voisinages.drop_duplicates(subset='index', keep="first")
voisinages = voisinages.set_index("index")
voisinages.index.name=None

In [11]:
df = df.merge(voisinages.loc[:, ['HISTORIQUE']], how='left', left_index=True, right_index=True)
df['HISTORIQUE']=np.where(df['HISTORIQUE_y'].isna(), df['HISTORIQUE_x'], df['HISTORIQUE_y'])
df = df.drop(columns=['HISTORIQUE_x', 'HISTORIQUE_y'])

In [12]:
df.columns

Index(['DATE ARRET', 'CDGI', 'QUARTIER', 'TYPE DE VOIE', 'PREFIXE de VOIE',
       'NOM de VOIE', 'Libellé VOIE', 'CVOIE', 'Personne', 'Dates', 'Dates2',
       'profession', 'profession2', 'profession3', 'LARGEUR', 'LONG', 'DETAIL',
       'ARRONDISSEMENT', 'STATUT PRESUME', 'DEBUT', 'FIN', 'HISTORIQUE'],
      dtype='object')

In [13]:
df.to_csv('Data/voies_paris_labelisé.csv', sep=';')